Models were automatically reconstructed from NCBI RefSeq accession codes with carveme.

For NJ4:

```
carve --refseq GCF_003014985.1 -o GEMs/NJ4.xml
```


In [1]:
from cobra.io import read_sbml_model

nj4 = read_sbml_model('GEMs/NJ4.xml')

In [2]:
from utils import model_validation

model_validation.energy_generation_cycle(nj4)

ATPM flux: 0.0


False

In [3]:
medium = nj4.medium

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


In [5]:
sol = nj4.optimize()

In [8]:
nj4.reactions.EX_btoh_e

AttributeError: DictList has no attribute or entry EX_btoh_e